# Description

In this file, we aim to categorize the data into four groups, namely 'Normal (no failure)', 'Failure_1 (failure in link one)', 'Failure_2 (failure in link two)', and 'Failure_3 (failure in link three)'. To this end, we use dataset 888_voa_3 (supervised) as the source dataset and all other datasets in the form of unsupervised. The objective is to develop a DANN (Domain Adversarial Neural Network) model to classify link failures in different datasets. We first implement the k-means clustring on each unsupervised dataset to generate psudo-labels relying on which we develop the DANN model.

# Import Libraries

In [1]:
import gc
import time
import copy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Function
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import TensorDataset, DataLoader

import os
os.environ['CURL_CA_BUNDLE'] = ''

import warnings

## Required Functions

In [3]:
def clone_gpu():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    gc.collect()

In [4]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [13]:
def prepare_train_test_data(X, y, batch_size=64, shuffle=True):
    input_dataset = TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long))
    data_loader = DataLoader(input_dataset, batch_size=batch_size, shuffle=shuffle)
    
    return data_loader

## Model

In [23]:
class GradReverse(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

    @staticmethod
    def grad_reverse(x, alpha=1.0):
        return GradReverse.apply(x, alpha)

class DANN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=4, drop_prob=0.3):
        super(DANN, self).__init__()
        hidden_dim_last = hidden_dim // 2
        hidden_dim_cd = input_dim
        self.input_dim = input_dim
        
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim_last),
            nn.ReLU(),
            nn.Linear(hidden_dim_last, hidden_dim_last),
            nn.ReLU(),
            nn.Linear(hidden_dim_last, hidden_dim_cd),
            nn.ReLU()
        )
        
        self.class_classifier = nn.Sequential(
            nn.Linear(hidden_dim_cd, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.LogSoftmax()
        )
        
        self.domain_classifier = nn.Sequential(
            nn.Linear(hidden_dim_cd, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2),
            nn.LogSoftmax(dim=1)
        )

        if torch.cuda.is_available():
            self.device = "cuda"
        elif torch.backends.mps.is_available():
            self.device = "mps"
        else:
            self.device = "cpu"
    
        self.to(self.device)

    def forward(self, x, alpha=1.0):
        features = self.feature_extractor(x)
        reverse_features = GradReverse.grad_reverse(features, alpha)
        class_output = self.class_classifier(features)
        domain_output = self.domain_classifier(reverse_features)
    
        return class_output, domain_output

## Train and Eval

In [26]:
def train(X_s, y_s, X_t, y_t, X_e, y_e):    
    warnings.filterwarnings('ignore')
    clone_gpu()
    set_seed(seed=42)

    num_epochs = 1000
    batch_size = 64
    lr = 0.001
    weight = 0.1
    ep_print = num_epochs // 10

    X_src = copy.deepcopy(X_s)
    y_src = copy.deepcopy(y_s)
    X_tgt = copy.deepcopy(X_t)
    y_tgt = copy.deepcopy(y_t)
    X_val = copy.deepcopy(X_e)
    y_val = copy.deepcopy(y_e)

    dataloader_source = prepare_train_test_data(X_src, y_src, batch_size=batch_size)
    dataloader_target = prepare_train_test_data(X_tgt, y_tgt, batch_size=batch_size)

    dict_unq_src = get_unique_label_data(X_src, y_src)
    dict_unq_tgt = get_unique_label_data(X_tgt, y_tgt)

    dataloader_src = []
    dataloader_tgt = []
    for key in dict_unq_src.keys():
        X_source = dict_unq_src[key]
        y_source = torch.ones(len(X_source), dtype=torch.long)*key
        dl_s = prepare_train_test_data(X_source, y_source, batch_size=batch_size)
        dataloader_src.append(dl_s)

        X_target = dict_unq_tgt[key]
        y_target = torch.ones(len(X_target), dtype=torch.long)*key
        dl_t = prepare_train_test_data(X_target, y_target, batch_size=batch_size)
        dataloader_tgt.append(dl_t)

    dataloader_val = prepare_train_test_data(X_val, y_val, batch_size=batch_size, shuffle=False)

    input_dim = X_src.shape[1]
    hidden_dim = 100
    output_dim = 4

    model = DANN(input_dim, hidden_dim)
    
    criterion_class = nn.NLLLoss().cuda()
    criterion_domain = nn.NLLLoss().cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    unique_labels = [i for i in range(output_dim)]

    loss_history = []
    start_time = time.time()
    for epoch in range(num_epochs):
        for dtl_src, dtl_tgt in zip(dataloader_src, dataloader_tgt):
            len_dataloader = min(len(dtl_src), len(dtl_tgt))
            data_zip = enumerate(zip(dtl_src, dtl_tgt))
    
            model.train()
            loss_list = []
            total_loss = 0
            for step, ((source_data, source_label), (target_data, _)) in data_zip:
                rho = float(step + epoch * len_dataloader) / (num_epochs * len_dataloader)
                alpha = 2. / (1. + np.exp(-10 * rho)) - 1

                optimizer.zero_grad()
                
                source_data = source_data.cuda()
                source_label = source_label.cuda()
                target_data = target_data.cuda()

                # source data forward
                domain_label = torch.zeros(len(source_data), dtype=torch.long).cuda()
                class_output, domain_output = model(source_data, alpha)
                loss_class_source = criterion_class(class_output, source_label)
                loss_domain_source = criterion_domain(domain_output, domain_label)
    
                # target data forward
                domain_label = torch.ones(len(target_data), dtype=torch.long).cuda()
                _, domain_output = model(target_data, alpha)
                loss_domain_target = criterion_domain(domain_output, domain_label)
    
                # loss
                loss_class = loss_class_source
                loss_domain = loss_domain_source + loss_domain_target
                loss = loss_class + loss_domain
                total_loss += loss
                
            loss = total_loss / step
            loss_list.append(loss.item())
            loss.backward()
            optimizer.step()

        accuracy = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in dataloader_val:
                inputs = inputs.cuda()
                predictions, _ = model(inputs)
                _, y_pred = torch.max(predictions.data, 1)
                acc = accuracy_score(y_pred.cpu(), labels)
                accuracy.append(acc * 100)
                
        accuracy_mean = np.mean(accuracy)
        mean_loss = np.mean(loss_list)
        loss_history.append(np.round(mean_loss, 2))
        if (epoch + 1) % ep_print == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy_mean: .2f}")
            
    runtime = time.time() - start_time

    return runtime, model

## Main

In [30]:
def evaluations(model, dt, end_ind, start_ind, batch_size=64):
    acc_cat = np.zeros((4, 4), dtype=np.float32)
    num_samples_acc = np.zeros(4, dtype=np.float32)
    acc = []

    model.eval()
    with torch.no_grad():
        X_val, y_val = get_data(dict_datasets[dt], end_ind, start_ind)
        val_loader = prepare_train_test_data(X_val, y_val, batch_size=batch_size, shuffle=False)
        for i, (inputs, labels) in enumerate(val_loader):    
            inputs = inputs.cuda()
            
            predictions, _ = model(inputs)
            _, y_pred = torch.max(predictions.data, 1)
            y_pred = y_pred.cpu().detach().numpy()

            acc.append(accuracy_score(y_pred, labels) * 100)

            for lbl_gt, lbl_pred in zip(labels, y_pred):
                acc_cat[lbl_gt][lbl_pred] += 1
                num_samples_acc[lbl_gt] += 1
        
    accuracy = np.mean(acc)
    for i in range(acc_cat.shape[0]):
        for j in range(acc_cat.shape[1]):
            acc_cat[i][j] = acc_cat[i][j] * 100 / num_samples_acc[i]

    return_list = [accuracy]
    for acc in acc_cat:
        return_list.append(acc)

    return return_list

In [ ]:
set_seed(seed=42)

results = []

X_src, y_src = get_data(dict_datasets[src_dataset], train_size)
models = []

for i, dt in enumerate(eval_total):
    print('Dataset:', dt)
    X_tgt_i, y_tgt_i = get_data(dict_datasets[dt], train_size)
    X_val_i, y_val_i = get_data(dict_datasets[dt], train_size + 100, train_size)
    runtime, model = train(X_src, y_src, X_tgt_i, y_tgt_i, X_val_i, y_val_i)
    res = evaluations(model, dt, val_size, train_size)
    models.append(model)

    new_row = [f'{dt}']
    for r in res:
        new_row.append(r)
    new_row.append(runtime)

    results.append(new_row)
    print(f'Accuracy: {res[0]}, Runitme: {runtime}')
    print('*****************************************')